# PerFedRec++ チュートリアル

このチュートリアルでは、プライバシーを保護するパーソナライズされた推薦システムである **PerFedRec++** の主要な概念と使い方を学びます。

## PerFedRec++とは？

PerFedRec++は、以下の3つの主要な課題に対処するために設計された、新しい連合推薦フレームワークです。
1.  **異質性とパーソナライゼーション**: ユーザーごとに異なる興味やデータ分布に対応し、パーソナライズされた推薦を提供します。
2.  **モデル性能の劣化**: プライバシー保護技術（差分プライバシーなど）を導入する際のモデル性能の低下を防ぎます。
3.  **通信のボトルネック**: 連合学習におけるサーバーとクライアント間の通信コストを削減します。

このチュートリアルを通じて、PerFedRec++がこれらの課題をどのように解決するのかを理解していきましょう。

## プロジェクトの構造

このリポジトリの主要なファイルとディレクトリは以下の通りです。

```
PerFedRec++/
├── conf/                # モデルの設定ファイル
│   └── PerFedRec_plus.conf
├── data/                # データローダーと前処理
├── dataset/             # データセット (例: yelp_test)
├── main.py              # メインの実行スクリプト
└── model/
    └── graph/
        └── PerFedRec_plus.py # PerFedRec++モデルの実装
```

- `main.py`: モデルの学習と評価を行うためのエントリーポイントです。
- `conf/`: 各モデルのハイパーパラメータなどの設定を管理します。
- `model/graph/`: `PerFedRec_plus.py` を含む、GNNベースの推薦モデルの実装が格納されています。
- `data/`: データの読み込みや変換を担当するモジュールが含まれます。


## モデルの実行方法

`main.py` を使ってモデルを学習・評価します。以下は基本的な実行コマンドの例です。

ターミナルで以下のコマンドを実行することで、Yelpデータセットを用いてPerFedRec++モデルを学習できます。

In [ ]:
# このセルはJupyter Notebook環境で直接実行するのではなく、
# ターミナルで実行するコマンドの例です。

!python PerFedRec++/main.py --model PerFedRec_plus --dataset yelp

### コマンドライン引数

主要な引数は以下の通りです。
- `--model`: 使用するモデル名 (`PerFedRec_plus`など)。
- `--dataset`: 使用するデータセット名 (`yelp`, `kindle`, `gowalla`)。
- `--emb`: 埋め込みの次元数 (デフォルト: 64)。
- `--pretrain_epoch`: 自己教師あり事前学習のエポック数 (デフォルト: 5)。

## 必要なライブラリのインストール

このプロジェクトを実行するには、いくつかのPythonライブラリが必要です。
以下のセルを実行して、必要なライブラリをインストールしてください。

In [ ]:
# PyTorch, scikit-learn, numpyなどをインストールします
%pip install torch scikit-learn numpy pandas

## PerFedRec++の主要な概念

PerFedRec++は、主に3つのモジュールで構成されています。それぞれのモジュールがどのように連携して、プライバシーを保護しつつ高精度な推薦を実現するのかを見ていきましょう。

### 1. 自己教師あり事前学習モジュール (Self-Supervised Pre-Training)

このモジュールは、本格的な学習の前にモデルを「準備」する役割を担います。目的は、モデルの性能を高め、学習の収束を速めることです。

**主なアイデア**: 連合学習のプライバシー保護メカニズム（クライアント選択、差分プライバシーなど）を、**データ拡張**として利用します。これにより、モデルはノイズに強く、より本質的なデータ表現を学習できます。

**仕組み**:
1.  **データ拡張**: ノードドロップアウト（一部のユーザーが学習に参加しないこと）やノイズ注入（データに意図的にノイズを加えること）などを用いて、同じデータから複数の異なる「ビュー」を生成します。
2.  **対照学習 (Contrastive Learning)**: 生成されたビューを使って学習します。「同じノードから生成されたビュー同士は似ているべき（ポジティブペア）」、「異なるノードのビューは似ていないべき（ネガティブペア）」という原則に基づき、モデルを学習させます。

**効果**:
- **性能向上**: 埋め込み表現がより豊かになり、推薦精度が向上します。
- **通信効率**: モデルが良い初期状態から学習を始められるため、収束が速くなり、通信回数が削減されます。

### 2. ユーザーサイドのローカル推薦ネットワーク (User-Side Local Recommendation Network)

各ユーザーのデバイス上で行われる、パーソナライズされたモデルの学習プロセスです。

**仕組み**:
1.  **ローカルGNN**: 各ユーザーは、自身のプライベートな利用履歴（どのアイテムを評価したかなど）を使って、ローカルでグラフニューラルネットワーク（GNN）を学習します。ユーザー自身の埋め込みはデバイス内に留まり、プライバシーが保護されます。
2.  **パーソナライズされた予測**: 最終的な推薦は、以下の3つのモデルを組み合わせて生成されます。
    - **グローバルモデル**: 全ユーザーのデータから学習した一般的なモデル。
    - **クラスタレベルモデル**: 似たような興味を持つユーザーグループのモデル。
    - **ローカルモデル**: ユーザー自身のデータで微調整されたモデル。

これにより、個人の好みを反映しつつ、他のユーザーの知識も活用した、バランスの取れた推薦が可能になります。

### 3. サーバーサイドのクラスタリングに基づく連合 (Server-Side Clustering-Based Federation)

中央サーバーが、ユーザーをグループ化し、効率的に学習を進めるための処理です。

**仕組み**:
1.  **ユーザークラスタリング**: サーバーは、各ユーザーから送られてきた埋め込み表現に基づき、似たようなユーザーをK個のクラスタに分類します（例: K-means法）。
2.  **モデルの集約**: サーバーは、全ユーザーから集約した**グローバルモデル**と、各クラスタ内で集約した**クラスタレベルモデル**の2種類を作成します。

このクラスタリングにより、ユーザーの多様性に対応しつつ、パーソナライズの精度を高めることができます。

## コードウォークスルー

ここでは、PerFedRec++の主要な処理の流れをコードと共に見ていきます。

### データの読み込み

まず、モデルが使用するデータをどのように読み込むかを見てみましょう。
データは `PerFedRec++/data/loader.py` と `PerFedRec++/data/data.py` にあるクラスによって処理されます。

以下は、Yelpデータセットの学習データを読み込み、最初の5行を表示する例です。

In [ ]:
import pandas as pd

# データセットのパス
train_path = 'PerFedRec++/dataset/yelp_test/train.txt'

# pandasを使ってデータを読み込みます。
# ファイルはスペース区切りで、ヘッダーはありません。
# 最初の列がユーザーID、残りがそのユーザーが評価したアイテムIDのリストです。
try:
    # 最初の行だけを読み込んで、構造を確認します
    with open(train_path, 'r') as f:
        first_line = f.readline().strip()
        parts = first_line.split(' ')
        user_id = parts[0]
        item_ids = parts[1:]
        print(f"ユーザーID: {user_id}")
        print(f"アイテムIDのリスト（最初の10件）: {item_ids[:10]}")
        print(f"ユーザー {user_id} は {len(item_ids)} 個のアイテムを評価しました。")

except FileNotFoundError:
    print(f"エラー: {train_path} が見つかりません。")

### 学習プロセスの概要

`main.py` は、設定を読み込み、`SELFRec.py` の `SELFRec` クラスを呼び出すことで学習プロセスを開始します。
`SELFRec` クラス内で、モデルの学習ループが実行されます。

以下に、学習プロセスの主要なステップを擬似コードで示します。

In [ ]:
# このセルは実際の実行コードではなく、学習プロセスの流れを理解するための擬似コードです。

class SimplifiedSELFRec:
    def __init__(self, config):
        # 1. 設定とデータに基づいてモデルを初期化
        self.recommender = self.initialize_recommender(config)
        # 2. 事前学習用の設定（もしあれば）
        self.pretrain_recommender = self.initialize_pretrainer(config)

    def execute(self):
        # 3. 自己教師あり事前学習を実行
        if self.pretrain_recommender:
            print("自己教師あり事前学習を開始...")
            self.pretrain_recommender.train()
            # 事前学習した重みをメインモデルにコピー
            self.copy_pretrained_weights()
            print("事前学習が完了しました。")

        # 4. メインの連合学習を実行
        print("連合学習を開始...")
        for epoch in range(config.max_epoch):
            # a. サーバーがユーザーをクラスタリング
            self.recommender.cluster_users()
            # b. 各クライアント（ユーザー）がローカルでモデルを更新
            self.recommender.client_update()
            # c. サーバーが各クライアントからの更新を集約
            self.recommender.server_aggregate()
            # d. モデルを評価
            self.recommender.evaluate()
            print(f"エポック {epoch+1} が完了しました。")
        print("連合学習が完了しました。")

# 実際のコードでは、これらの処理は `PerFedRec++/main.py` と `PerFedRec++/SELFRec.py` で行われます。


### モデルアーキテクチャ (`PerFedRec_plus.py`)

モデルの中核は `PerFedRec++/model/graph/PerFedRec_plus.py` に実装されています。
このファイル内の `PerFedRec_plus` クラスが、前述の連合学習プロセスを具体的に実行します。

特に重要なのは `train()` メソッドで、以下の処理をオーケストレーションします。
- **クライアント選択**: 学習に参加するユーザーを選択します。
- **ローカル更新**: 各クライアントが自身のデータでモデルを学習します（`client_update`）。
- **サーバー集約**: サーバーがクライアントからの更新をまとめ、グローバルモデルとクラスタレベルモデルを更新します（`server_aggregate`）。

GNNには `LightGCN` のような軽量なモデルがベースとして使われており、効率的なグラフベースの推薦を実現しています。

## まとめ

このチュートリアルでは、PerFedRec++の基本的な概念と使い方を学びました。

- **自己教師あり事前学習**でモデルの初期状態を改善し、
- **クライアントサイドでのパーソナライズ**と**サーバーサイドでのクラスタリング**を組み合わせることで、

プライバシーを保護しながら、ユーザー一人ひとりに最適化された高精度な推薦を実現しています。

より詳細な実装については、`PerFedRec++/` ディレクトリ内の各Pythonファイルをご参照ください。